In [1]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = True)

Mounted at /content/drive


#### Import dependencies

In [3]:
pip install mcfly

In [4]:
import os
import statistics
import csv
import itertools
import tensorflow as tf
import keras
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical
from keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import mcfly
import os
import statistics
import csv
import itertools
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sn
from numpy import mean, std

### PAMAP2

In [5]:
cd "/content/drive/MyDrive/PAMAP2_Dataset"

/content/drive/MyDrive/PAMAP2_Dataset


In [6]:
df = pd.read_pickle('pamap2.df')

#### Create sliding windows from samples

In [7]:
window_size = 200
stride = 50
frames = []
for i in tqdm(range(0, len(df)-window_size, stride)):
    window = df.iloc[i:i+window_size]
    if window['activity_id'].nunique() == 1:
      frames.append(window)

<ipython-input-7-278bc114e726>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(df)-window_size, stride)):


  0%|          | 0/38726 [00:00<?, ?it/s]

In [8]:
X_list = []
y_list = []
relevant_columns = [col for col in frames[0].columns if col not in ['subject_id', 'activity_id', 'time_ste[]']]

#replace label with activity
for frame in tqdm(frames):
    X_list.append(frame[relevant_columns].values)
    y_list.append(frame.iloc[0]['activity_id'])

<ipython-input-8-29c4dda3d82c>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(frames):


  0%|          | 0/38310 [00:00<?, ?it/s]

In [9]:
X = np.array(X_list)
y = np.array(to_categorical(y_list))

In [10]:
X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=42)

print("X_train: ",X_train.shape)
print("X_val: ",X_val.shape)
print("X_test: ",X_test.shape)

print("Y_train: ",y_train.shape)
print("Y_val: ",y_val.shape)
print("Y_test: ",y_test.shape)

X_train:  (22986, 200, 21)
X_val:  (7662, 200, 21)
X_test:  (7662, 200, 21)
Y_train:  (22986, 25)
Y_val:  (7662, 25)
Y_test:  (7662, 25)


In [11]:
#fit scaler on train
scaler = StandardScaler()
train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_scaled = scaler.fit_transform(train_reshaped)
X_train = X_train_scaled.reshape(X_train.shape)

val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_val_scaled = scaler.transform(val_reshaped)
X_val = X_val_scaled.reshape(X_val.shape)

test_reshaped = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled = scaler.transform(test_reshaped)
X_test = X_test_scaled.reshape(X_test.shape)

In [12]:
n_steps, n_length,n_features = 4,50,21
trainX = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
valX = X_val.reshape((X_val.shape[0], n_steps, n_length, n_features))
testX = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))

In [13]:
from scipy.stats import entropy

js_divergences = np.zeros(trainX.shape[1])
for i in range(trainX.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(trainX[:, i])), len(np.unique(testX[:, i])))
    p, _ = np.histogram(trainX[:, i], bins=bins, density=True)
    q, _ = np.histogram(testX[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.3867837427756243


### MHEALTH Dataset

In [22]:
cd "/content/drive/MyDrive/MHEALTHDATASET/"

/content/drive/MyDrive/MHEALTHDATASET


In [55]:
df_mhealth = pd.read_pickle('mhealth_cnn.pkl')

#### sliding windows

In [56]:
df = df_mhealth
window_size = 200
stride = 50
frames = []
for i in tqdm(range(0, len(df)-window_size, stride)):
    window = df.iloc[i:i+window_size]
    if window['activity'].nunique() == 1:
      frames.append(window)

<ipython-input-56-285c3bb12267>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(0, len(df)-window_size, stride)):


  0%|          | 0/24311 [00:00<?, ?it/s]

In [57]:
X_list = []
y_list = []
#relevant_columns = [col for col in frames.columns if col not in ['subject_id', 'activity_id']]
relevant_columns = [col for col in frames[0].columns if col not in ['subject', 'activity', 'time_ste[]']]

#for each frame replace label with activity
for frame in tqdm(frames):
    X_list.append(frame[relevant_columns].values)
    y_list.append(frame.iloc[0]['activity'])

<ipython-input-57-3f1e8c77d07e>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(frames):


  0%|          | 0/23310 [00:00<?, ?it/s]

In [58]:
X = np.array(X_list)
y = np.array(to_categorical(y_list))

#### contaminated split

In [59]:
X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.6, random_state=429)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=429)

In [60]:
#fit scaler on train
scaler = StandardScaler()
train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_train_scaled = scaler.fit_transform(train_reshaped)
X_train = X_train_scaled.reshape(X_train.shape)

In [61]:
#transform test, val based on train scaler
val_reshaped = X_val.reshape(-1, X_val.shape[-1])
X_val_scaled = scaler.transform(val_reshaped)
X_val = X_val_scaled.reshape(X_val.shape)

test_reshaped = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled = scaler.transform(test_reshaped)
X_test = X_test_scaled.reshape(X_test.shape)

In [62]:
print("X_train: ",X_train.shape)
print("X_val: ",X_val.shape)
print("X_test: ",X_test.shape)

print("Y_train: ",y_train.shape)
print("Y_val: ",y_val.shape)
print("Y_test: ",y_test.shape)

X_train:  (9324, 200, 23)
X_val:  (6993, 200, 23)
X_test:  (6993, 200, 23)
Y_train:  (9324, 13)
Y_val:  (6993, 13)
Y_test:  (6993, 13)


In [63]:
n_steps, n_length,n_features = 4,50,23
trainX = X_train.reshape((X_train.shape[0], n_steps, n_length, n_features))
valX = X_val.reshape((X_val.shape[0], n_steps, n_length, n_features))
testX = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))

In [49]:
history = model.fit(trainX,y_train, epochs=8, validation_data=(valX, y_val), verbose=1 , batch_size  = 128 )

Epoch 1/8
73/73 [==============================] - 23s 248ms/step - loss: 1.1709 - acc: 0.7196 - val_loss: 0.7893 - val_acc: 0.7522
Epoch 2/8
73/73 [==============================] - 15s 207ms/step - loss: 0.5677 - acc: 0.7917 - val_loss: 0.3791 - val_acc: 0.8443
Epoch 3/8
73/73 [==============================] - 29s 407ms/step - loss: 0.3364 - acc: 0.8582 - val_loss: 0.2962 - val_acc: 0.8832
Epoch 4/8
73/73 [==============================] - 28s 381ms/step - loss: 0.2665 - acc: 0.8932 - val_loss: 0.2350 - val_acc: 0.9048
Epoch 5/8
73/73 [==============================] - 21s 286ms/step - loss: 0.2390 - acc: 0.9022 - val_loss: 0.2364 - val_acc: 0.9033
Epoch 6/8
73/73 [==============================] - 15s 211ms/step - loss: 0.2189 - acc: 0.9084 - val_loss: 0.2204 - val_acc: 0.9151
Epoch 7/8
73/73 [==============================] - 15s 211ms/step - loss: 0.1977 - acc: 0.9217 - val_loss: 0.1927 - val_acc: 0.9228
Epoch 8/8
73/73 [==============================] - 15s 207ms/step - loss: 0.

In [ ]:
from scipy.stats import entropy
import numpy as n

# Compute Jensen-Shannon divergence for each feature
js_divergences = np.zeros(trainX.shape[1])
for i in range(trainX.shape[1]):
    # Compute histograms with the same bins for both X_train and X_test
    bins = max(len(np.unique(trainX[:, i])), len(np.unique(testX[:, i])))
    p, _ = np.histogram(trainX[:, i], bins=bins, density=True)
    q, _ = np.histogram(testX[:, i], bins=bins, density=True)
    m = 0.5 * (p + q)
    js_divergences[i] = 0.5 * (entropy(p, m) + entropy(q, m))

avg_js_divergence = np.mean(js_divergences)
print("Average Jensen-Shannon divergence:", avg_js_divergence)

Average Jensen-Shannon divergence: 0.2639160243091124


### ML model

In [47]:
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=96, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Dropout(0.06)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64 , return_sequences= True ))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation = 'softmax'))


In [ ]:
plot_model(model, to_file='model1_plot.png', show_shapes=True,rankdir = 'TB',expand_nested = False,  show_layer_names=False, show_dtype =False)

In [48]:
model.compile(Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['acc'])

In [ ]:
history = model.fit(trainX,y_train, epochs=8, validation_data=(valX, y_val), verbose=1, batch_size=128)

In [ ]:
model.save("cnn_contaminated.h5")

### predict on test

In [51]:
loaded_model = tf.keras.models.load_model("cnn_contaminated.h5")

In [52]:
lstm_y_test_pred = loaded_model.predict(testX)
y_test_argmax = y_test.argmax(axis=1)
y_test_pred_argmax = lstm_y_test_pred.argmax(axis=1)
og_test = np.argmax(y_test, axis=1)

219/219 [==============================] - 7s 27ms/step


In [53]:
print("Smartphone Results with CNN-GRU for Test Set\n\n" + classification_report(y_test_pred_argmax,og_test , digits= 4))

Smartphone Results with CNN-GRU for Test Set

              precision    recall  f1-score   support

           0     0.9199    0.9819    0.9499      4794
           1     0.9699    0.7031    0.8152       229
           2     0.9136    0.7220    0.8065       205
           3     1.0000    0.7468    0.8550       233
           4     0.9693    0.8876    0.9267       178
           5     0.9112    0.8324    0.8701       185
           6     0.9530    0.8554    0.9016       166
           7     0.9273    0.8361    0.8793       183
           8     0.9940    0.7661    0.8653       218
           9     0.9239    0.9189    0.9214       185
          10     1.0000    0.8407    0.9134       182
          11     0.9412    0.8649    0.9014       185
          12     0.7358    0.7800    0.7573        50

    accuracy                         0.9275      6993
   macro avg     0.9353    0.8258    0.8741      6993
weighted avg     0.9298    0.9275    0.9254      6993

